In [3]:
!pip install cupy-cuda12x

In [23]:
!pip install copy

ERROR: Could not find a version that satisfies the requirement copy (from versions: none)
ERROR: No matching distribution found for copy


In [25]:
import numpy as np
import cupy as cp
from copy import deepcopy

In [6]:
def lspace(start, stop, n):
    res = []
    counter = start
    spacing = str((stop - start) / n)
    spacingdecs = len(spacing) - 2
    spacing = float(spacing)
    for _ in range(n):
        res.append(counter)
        counter += spacing
        counter = np.float32(counter)
        counter = np.around(counter, spacingdecs)
    res.append(counter)
    return res

In [104]:
l = -2
r = 2
d = -2
u = 2
npax = 2000
npay = 2000
spcsHor = lspace(int(l), int(r), npax)
spcsVer = lspace(int(d), int(u), npay)
fullspcs = [tuple([x, y, 0]) for x in spcsHor for y in spcsVer]
points = np.array([[0, complex(x, y)] for x in spcsHor for y in spcsVer], dtype=cp.complex64)
points = cp.asarray(points, dtype=cp.complex64)
outSince = cp.asarray(cp.zeros(len(fullspcs)))
prev2 = cp.asarray([cp.zeros(len(fullspcs), dtype=cp.complex64)] * 2)

In [85]:
def updOutSince():
    global outSince
    outSince += (cp.absolute(points[:, 0]) > 2)

In [86]:
def orbitStep():
    prev2[0] = cp.copy(prev2[1])
    prev2[1] = cp.copy(points[:, 0])

    points[:, 0] **= 2
    points[:, 0] += points[:, 1]

    #def: move points way too far closer but still off screen to avoid overflow
    # self.points[:, 0][abs(self.points[:, 0]) > 15] = self.points[:, 0][abs(self.points[:, 0]) > 15]
    #problem: points in the negatives get dragged across the screen
    #solution: 10-unit vector

    points[:, 0] = cp.where(cp.absolute(points[:, 0]) > 100, points[:, 0] / cp.absolute(points[:, 0]) * 10, points[:, 0])

    updOutSince()

In [105]:
for _ in range(200):
  orbitStep()

In [106]:
points

array([[54.652367  +80.40455j  , -2.         -2.j       ],
       [86.05155   +45.403732j , -2.         -1.998j    ],
       [-0.86981326 +9.962099j , -2.         -1.996j    ],
       ...,
       [ 7.2858286  +6.8495755j,  2.         +1.996j    ],
       [ 7.2464175  +6.8912573j,  2.         +1.998j    ],
       [ 7.362704   -6.7668734j,  2.         +2.j       ]],
      dtype=complex64)

In [107]:
np.save('./points.npy', cp.asnumpy(points))